# SAM2 Surface Defect Detection Demo

This notebook demonstrates how to use SAM2 for detecting surface defects in automotive/industrial images.

**Detectable Defects:**
- 🎨 Paint defects (color variations)
- 🔍 Surface contamination (spots, stains)
- 🦠 Corrosion/rust patterns
- 💧 Water spots and mineral deposits

**Requirements:**
- GPU runtime recommended for faster processing
- Internet connection for downloading SAM2 models


## 1. Setup and Installation

First, let's install all the required dependencies:


## 1.5. Memory Management Setup (Important!)

For processing large images without running out of GPU memory:


In [ ]:
# Set environment variables for better memory management
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear any existing GPU cache
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ GPU cache cleared")

# Optional: Set memory fraction (uncomment if still having memory issues)
# torch.cuda.set_per_process_memory_fraction(0.8)  # Use only 80% of GPU memory

print("✅ Memory management configured!")


In [ ]:
# Install PyTorch with CUDA support (for GPU acceleration)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install SAM2 from Facebook Research
!pip install git+https://github.com/facebookresearch/segment-anything-2.git

# Install other required packages
!pip install opencv-python pillow matplotlib numpy tqdm

print("✅ All dependencies installed successfully!")


## 2. Check GPU Availability


In [ ]:
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ CUDA not available. Consider enabling GPU runtime for faster processing.")
    print("Go to Runtime -> Change runtime type -> Hardware accelerator -> GPU")


## 3. Upload the SAM2 Surface Defect Detection Script

Upload the `sam2_surface_defect_detector.py` file to your Colab environment:


In [ ]:
from google.colab import files
import os

print("Please upload the sam2_surface_defect_detector.py file:")
uploaded = files.upload()

# Check if the script was uploaded
if 'sam2_surface_defect_detector.py' in uploaded:
    print("✅ Script uploaded successfully!")
    print(f"File size: {len(uploaded['sam2_surface_defect_detector.py'])} bytes")
else:
    print("❌ Please upload the sam2_surface_defect_detector.py file")


## 4. Upload Test Image

Upload an image that you want to analyze for surface defects:


In [ ]:
print("Please upload an image for surface defect detection:")
uploaded_images = files.upload()

# Get the first uploaded image
image_path = None
for filename in uploaded_images.keys():
    image_path = filename
    break

if image_path:
    print(f"✅ Image uploaded: {image_path}")
    print(f"File size: {len(uploaded_images[image_path])} bytes")
    
    # Display the uploaded image
    from PIL import Image
    import matplotlib.pyplot as plt
    
    img = Image.open(image_path)
    plt.figure(figsize=(10, 6))
    plt.imshow(img)
    plt.title(f"Uploaded Image: {image_path}")
    plt.axis('off')
    plt.show()
    
    print(f"Image dimensions: {img.size}")
else:
    print("❌ Please upload an image file")


## 5. Initialize the Detector

Now let's import our surface defect detector:


In [ ]:
# Import the detector class
from sam2_surface_defect_detector import SAM2SurfaceDefectDetector, visualize_results, print_detection_summary

# Initialize the detector
print("Initializing SAM2 Surface Defect Detector...")
print("This may take a few minutes on first run as it downloads the SAM2 model.")
print()

detector = SAM2SurfaceDefectDetector(
    sam2_model="facebook/sam2-hiera-tiny",  # Use the smallest/fastest model for demo
    device="cuda" if torch.cuda.is_available() else "cpu"
)

print("✅ Detector initialized successfully!")


## 6. Run Surface Defect Detection

Now let's analyze the uploaded image for surface defects:


In [ ]:
if image_path:
    print(f"Analyzing image: {image_path}")
    print("This may take a few minutes depending on image size and hardware...")
    print()
    
    # Clear GPU cache before processing
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Run detection with memory-efficient settings
    # For very large images (>2000px), use max_image_size=512 or 768
    # For normal images, use max_image_size=1024 (default)
    detections = detector.detect_surface_defects(image_path, max_image_size=1024)
    
    # Print summary
    print_detection_summary(detections)
    
else:
    print("❌ No image uploaded. Please run the image upload cell first.")


### 6.1. If You Get Memory Errors

If you encounter "CUDA out of memory" errors, try this cell with smaller image processing size:


In [ ]:
# If you got memory errors, try with smaller image size
if image_path:
    print("🔧 Trying with memory-optimized settings...")
    
    # Clear all GPU memory
    torch.cuda.empty_cache()
    
    # Try with much smaller image size for memory-constrained environments
    try:
        detections = detector.detect_surface_defects(image_path, max_image_size=512)
        print_detection_summary(detections)
        print("✅ Success with smaller image size!")
    except Exception as e:
        print(f"❌ Still failed with 512px: {e}")
        print("Trying with even smaller size...")
        
        # Last resort: very small image
        try:
            detections = detector.detect_surface_defects(image_path, max_image_size=256)
            print_detection_summary(detections)
            print("✅ Success with very small image size!")
        except Exception as e:
            print(f"❌ Failed even with 256px: {e}")
            print("Your image might be too complex or GPU memory is insufficient.")
            print("Consider using CPU mode or a simpler image.")
else:
    print("❌ No image uploaded.")


## 7. Visualize Results

Let's create a visual representation of the detected defects:


In [ ]:
if image_path and 'detections' in locals():
    print("Creating visualization...")
    
    # Create and display the visualization
    fig = visualize_results(image_path, detections, save_path=None)
    
    # Also save the result
    output_filename = f"defects_detected_{image_path.split('.')[0]}.png"
    fig.savefig(output_filename, dpi=150, bbox_inches='tight')
    
    print(f"\n✅ Visualization saved as: {output_filename}")
    
    # Offer to download the result
    files.download(output_filename)
    
else:
    print("❌ Please run the detection step first.")


## 🎉 Conclusion

You have successfully used SAM2 for surface defect detection! Here's what we accomplished:

✅ **Installed** SAM2 and all dependencies  
✅ **Loaded** and initialized the surface defect detector  
✅ **Analyzed** images for various types of surface defects  
✅ **Visualized** results with bounding boxes and labels  
✅ **Generated** detailed reports and statistics  

### Next Steps:

1. **Fine-tuning**: Adjust detection thresholds in the script for your specific use case
2. **Custom Training**: Train SAM2 on your specific defect dataset
3. **Integration**: Integrate this detection pipeline into your production system
4. **Optimization**: Use model quantization or pruning for faster inference

### Key Features of This Implementation:

- 🔍 **Multiple Defect Types**: Paint defects, contamination, corrosion, water spots
- 🚀 **GPU Acceleration**: Automatic CUDA detection and usage
- 📊 **Detailed Analysis**: Confidence scores, locations, and defect characteristics
- 🎨 **Rich Visualization**: Color-coded bounding boxes with labels
- 📦 **Batch Processing**: Handle multiple images efficiently
- 🔧 **Flexible Models**: Support for different SAM2 model sizes

Feel free to experiment with different images and model configurations!
